# Interrogation du point d'accès SPARQL de DBPedia (en anglais)


Exploration des données liées aux 'économistes'

Définition de économiste.


Après une inspection manuelle de quelques économistes connus.

## Préparation

### Importer les librairies à utiliser

In [5]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pprint
import csv
import ast

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [3]:
### Importer un module de fonctions crée ad hoc
## ATTENTION : le fichier 'sparql_functions.py' doit se trouver 
#   dans le même dossier que le présent carnet Jupyter afin que l'importation
#   fonctionne correctement
import sparql_functions as spqf

In [4]:
### Recharger le module après modification des fonctions dans le fichier du module
# désactivé #  reload(spqf)

## Création de la base de données SQLite

Tutoriel SQL pour SQLite : https://www.sqlitetutorial.net/sqlite-select/

Afin de créer la base de données:
* créer tout d'abord (ou vérifier qu'existe déjà) un dossier appelé 'data'
* décommenter les parties commentées et exécuter ensuite les scripts de créations des tables ci-dessous
* inspecter les nouvelles tables avec un [client graphique SQLite tel DBeaver](http://phn-wiki.ish-lyon.cnrs.fr/doku.php?id=intro_histoire_numerique:modelisation_bases_donnees#clients_graphiques_pour_sqlite)


### Création de la base de données

In [4]:
### La connextion à une base de données SQLite créé la base de données si elle n'est pas encore existante, 
#  et ce dans le dossier indiqué

cn = sql.connect('data/sparql_queries.db')

In [5]:
## Un curseur est un objet qui permet d'exécuter des requêtes sur la base des données en les isolant proprement
c = cn.cursor()

##  https://www.sqlite.org/lang_datefunc.html
# on vérifie ainsi qu'on est bien connecté à la base et on a l'heure locale actuelle
c.execute("SELECT datetime('now', 'localtime')")
print(c.fetchone())

('2021-05-17 17:34:19',)


In [6]:
## On ferme la base de données
cn.close()

### Production des tables de la base de données

In [51]:
db = 'data/sparql_queries.db'   ### 'data/database.db'

In [ ]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# STOP # c.execute('''DROP TABLE query;''')

c.execute('''CREATE TABLE query (pk_query INTEGER PRIMARY KEY,
label VARCHAR (100),
description TEXT,
project VARCHAR (150),
sparql_endpoint VARCHAR (250),
query TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# désactivé # cn.commit()
cn.close()

In [ ]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# STOP # c.execute('''DROP TABLE result;''')

c.execute('''CREATE TABLE result (pk_result INTEGER PRIMARY KEY,
fk_query INTEGER REFERENCES "query" (pk_query) MATCH SIMPLE,
description TEXT,
result TEXT,
insert_data_into TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# désactivé # cn.commit()
cn.close()

In [185]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'entity'
# STOP #  c.execute('''DROP TABLE entity''')

c.execute('''CREATE TABLE entity
             (pk_entity INTEGER PRIMARY KEY, local_authority BOOLEAN DEFAULT TRUE, 
             fk_same_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, entity_class TEXT, source TEXT,
             fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             entity_std_name TEXT, notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, entity_class, source))''')

# Save (commit) the changes 
# STOP #  cn.commit()

cn.close()

In [207]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'property'
# STOP # c.execute('''DROP TABLE property''')

c.execute('''CREATE TABLE property
             (pk_property INTEGER PRIMARY KEY, 
             fk_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, property TEXT, value TEXT, 
             source TEXT, quality INTEGER, fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             additional_property_1 TEXT, value_1 TEXT,
             additional_property_2 TEXT, value_2 TEXT,
             additional_property_3 TEXT, value_3 TEXT,
             notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, property, value, source))''')

# Save (commit) the changes 
# STOP #  cn.commit()

cn.close()

### Versionnement manuel de la base de donnée des requêtes

Cette opération créé une copie de la base de données identifiée par un _timestamp_. À utiliser avant toute opération significative sur la base de données (modification de la structure, imports par des scripts Python, etc.)

In [84]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210418_113708'

In [85]:
### Définir les adresses des fichiers, l'existant et celui à créer
db = 'data/sparql_queries.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [86]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afint de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210418_113709.sqlite'

## Explorer Wikipedia / DBPedia

DBPedia est une base de données au format RDF réalisée par extraction de données semistructurées et structurées à partir (principalement) des Infobox et du résumé des pages de Wikipedia.

Les versions de DBPedia sont liées aux versions linguistiques de Wikipedia et dépendent de l'activité de 'chapitres' nationaux de DBPedia qui effectuent ces extractions. Il existe une [communauté (un 'chapitre')](https://www.data.gouv.fr/fr/datasets/dbpedia-fr-le-chapitre-francophone-de-dbpedia/) et une [version francophone](http://fr.dbpedia.org/).

Nous explorerons ici la version anglaise qui est la plus riche mais les différentes versions sont connectées, et on peut profiter de ces versions.

### Wikipedia: Les économistes, page "Economist"

*  https://en.wikipedia.org/wiki/Economist
*  https://en.wikipedia.org/wiki/Economics


### DBPedia : Economist

*  https://dbpedia.org/page/Economist  (an instance of the class [Person Function](https://dbpedia.org/ontology/PersonFunction) )
*  https://dbpedia.org/ontology/Economist  (a class)


*  https://dbpedia.org/page/Economics


### DBPedia SPARQL Endpoint

*  https://dbpedia.org/sparql
*  https://live.dbpedia.org/sparql (mis à jour en permanence, à utiliser pour les requêtes fédérées, i.e. la clause SERVICE)

Documentation : [SPARQL 1.1 Query Language](http://www.w3.org/TR/sparql11-query/)

### Keynes dans DBPedia

*  https://en.wikipedia.org/wiki/John_Maynard_Keynes
*  https://dbpedia.org/page/John_Maynard_Keynes



## Premières requêtes SPARQL

### Utiliser le point d'accès SPARQL de DBPedia

* Interroger le point d'accès de DBPedia 
  * Ouvrir dans une autre fenêtre du navigateur:
  * https://dbpedia.org/sparql
* Copier-coller le texte de la requête ci-dessous dans le navigateur:
* Exécuter et inspecter le résultat


NB : exécuter des requêtes directement sur le point SPARQL DBPedia permet de tester pour vérifier si elles fonctionnent correctement



In [ ]:
###  Définir la variable contenant la requête SPARQL
# Requête : toutes les propriétés concernant Keynes

q_1 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p ?o WHERE {
dbpedia:John_Maynard_Keynes ?p ?o
}

"""

In [ ]:
###  Requête: toutes les propriétés ayant un texte en anglais en objet


q_2 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p ?o WHERE {
dbpedia:John_Maynard_Keynes ?p ?o.
FILTER ( lang(?o) = "en" ) 
}

"""

In [7]:
###  Requête: compter et regrouper les propriétés

q_3 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p (COUNT(*) as ?eff) WHERE {
dbpedia:John_Maynard_Keynes ?p ?o.
}
GROUP BY ?p
ORDER BY DESC(?eff)

"""

In [ ]:
### Requête: inspecter le contenu des différentes propriétés, en les analysant une par une

# Astuce: ouvrir une nouvelle fenêtre du point d'accès SPARQL DBPedia
# A partir du résultat de la requête précédente, copier coller les proriétés et exécuter

q_4 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?o WHERE {
dbpedia:John_Maynard_Keynes <http://dbpedia.org/ontology/birthPlace> ?o.
}

"""

In [ ]:
### Requête : compter les instances (personnes) de la classe dbo:Economist

q_5 = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>

SELECT  (COUNT(*) AS ?freq)
WHERE
{ ?person  a dbo:Economist.
}

"""

In [7]:
### Pour toute nouvelle requête avant stockage

my_query = """
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>

SELECT  (count(*) AS ?eff)
WHERE

  {   { ?person  dbp:profession  dbr:Economist }
    UNION
      { ?person  dbp:occupation  dbr:Economist }
  }
  """

   

    

In [97]:
my_query = """PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  dbc: <http://dbpedia.org/resource/Category:>

SELECT DISTINCT  ?person     # (count(*) AS ?eff)
WHERE
  {  
      { ?person  dbo:wikiPageWikiLink dbc:20th-century_economists}
  }
  ORDER BY ?person
  """


### Exécuter les mêmes requêtes depuis le carnet Jupyter

In [102]:
### Librairie SPARQLWrapper, indiquer sur quel point d'accès SPARQL exécuter la requête
sparql = SPARQLWrapper("https://dbpedia.org/sparql") ##, returnFormat=RDFXML) # "https://live.dbpedia.org/sparql"

In [103]:
### Mettre en argument de la fonction setQuery, qui définit la requête, le nom de la variable correspondante
#  On définit le format du resultat (JSON), puis on effectue la requête en ligne 
# et on inspecte le type de résultat: le résultat en JSON a été transformé en une variable 'dictionnaire'

sparql.setQuery(my_query)
sparql.setReturnFormat(JSON)
rc = sparql.queryAndConvert()
type(rc)

dict

In [ ]:
### Inspecter le résultat complet
# Activer 'enable scrolling outputs' dans le carnet Jupyter (click droit sur le résultat)
rc

In [105]:
### En alternative à la requête précédente, transformer le résultat JSON en texte et inspecter les 500 premiers caractères
str(rc)[:500]

"{'head': {'link': [], 'vars': ['person']}, 'results': {'distinct': False, 'ordered': True, 'bindings': []}}"

#### Documentation JSON / Python _dictionary_

* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/tutorial/datastructures.html#dictionaries


In [95]:
### Nombre de lignes du résultat, utiliser la structure JSON (dictionnaire en Python)
len(rc['results']['bindings'])

0

In [96]:
# Inspecter les trois premières lignes du résultat
i = 0
for l in rc['results']['bindings']:
    if i < 3:
        print(l)
        i += 1

In [ ]:
r = [l for l in spqf.sparql_result_to_list(rc)]
r[:3]

In [88]:
sq = spqf.sparql_result_to_list(rc)

In [106]:
### L'économiste Keynes n'est pas de type 'économiste' dans DBPedia !
a = [l for l in sq if 'Keynes' in l]
len(a)

0

## Effectifs et propriétés de 'economist' et 'economics'

### Stocker les requêtes dans une base de données SQLite


*  Afin de ne pas surcharger le carnet, les requêtes SPARQL et les résultats sont stockées dans une base de données SQLite.
*  Elle se trouve dans le dossier _data/sparql_queries.db_ .
*  On peut l'ouvrir et l'éditer avec le logiciel SQLiteStudio, cf. les indications de [cette page](http://phn-wiki.ish-lyon.cnrs.fr/doku.php?id=intro_histoire_numerique:modelisation_bases_donnees)


#### Manière de procéder

* Préparer la requête directement dans l'éditeur du point d'accès SPARQ original, dans ce cas: https://dbpedia.org/sparql
* Si nécessaire, valider et formater les requêtes dans ces plateformes:
  *  https://linked.bodc.ac.uk/validate/query
  *  http://sparql.org/query-validator.html
* Copier ensuite la requête dans une nouvelle ligne de la base de données, table _query_
* Ajouter un titre, un descriptif, l'adresse du point d'accès SPARQL dans la nouvelle ligne de la base de données
* Choisir la ligne à exécuter et retenir les numéro de sa clé primaire (_pk_query_)


In [ ]:
### Inspecter les requêtes existantes

# connnexion à la base de données
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT pk_query, label, description, sparql_endpoint FROM query')
r_all = c.fetchall()

# fermer la connexion
conn.close()
r_all

## Requête SPARQL à partir d'une requête stockée dans la base de données

In [7]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = 8

# connnexion à la base de données
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
# rc


In [8]:
print(rc[2] +  "\n-----\n" + rc[4] +  "\n-----\n" + rc[5])

Cherche toutes les propriétés des personnes classées dans l'ontologie DBPedia comme dbo:Economist ou associées avec des propriétés significatives au concept dbr:Economist


7 mai 2021 : ajouté yago:Economist110043643

Le volume des personnes retrouvées est plus que doublé et celui de leurs propriétés également
-----
https://dbpedia.org/sparql
-----
PREFIX  dbo:  <http://dbpedia.org/ontology/>
PREFIX  dbp:  <http://dbpedia.org/property/>
PREFIX  dbr:  <http://dbpedia.org/resource/>
PREFIX  yago: <http://dbpedia.org/class/yago/>

SELECT  ?p ?direction (count(*) AS ?freq)
WHERE
  {   { ?person  ?p  ?o
        BIND("out" AS ?direction)
      }
    UNION
      { ?s  ?p  ?person
        BIND("in" AS ?direction)
      }
    {   { ?person  a                     dbo:Economist }
    UNION
      { ?person a yago:Economist110043643}    
      UNION
        { ?person  dbp:profession  dbr:Economist }
      UNION
        { ?person  dbp:occupation  dbr:Economist }
      UNION
        { ?person  dbp:fi

### Exécuter les requêtes SPARQL en utilisant les fonctions-utilisateur

In [9]:
### Exécuter la requête SPARQL enveloppée par cette fonction, 
# elle se trouve dans le biliothèque-utilisateur _sparql_functions.py_
# le premier paramètre correspond au point d'accès SPARQL, le deuxième à la requête
qr = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [10]:
# Nombre de lignes de la réponse
len(qr['results']['bindings'])

1460

In [ ]:
# Inspecter les dix premières lignes
i = 0
for l in qr['results']['bindings']:
    if i < 10:
        print(l)
        i += 1

In [15]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(qr)  if 'ontology' in l[0] ]
# r

In [ ]:
# inspecter une seule ligne de la liste (de listes)
print(len(r))
r[:20]

### Stocker le résultat dans la base de données SQLite

Le résultat de la requête est stocké tel quel, en entier, dans une ligne de la base de données. L'horodatage est ajouté afin de situer le résultat dans le temps.

In [16]:
### OPTIONNEL : si souhaité, stocker la réponse du point d'accès SPARQL dans la table 'result' de la base de données 

conn = sql.connect(original_db)
c = conn.cursor()
values = (pk_query, str(qr))

# https://www.techonthenet.com/sqlite/functions/now.php
c.execute("INSERT INTO result (fk_query, result) VALUES (?,?)", values)
# valider l'insertion et fermer la base de données
## DESACTIVÉ !!! ##  conn.commit()
conn.close()

In [7]:
### Inspecter le résultat d'une requête après insertion, une fois l'insert ci-dessus effectué

# Définir la ligne de la base de données à chercher (inspécter préalablement la base de données)
pk_result = ['11']

# connnexion à la base de données
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM result WHERE pk_result = ?', pk_result)
result_q = c.fetchone()

# fermer la connexion
conn.close()
# result_q[3]

In [8]:
### Transform string to dict
## Doc.:
# https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
d = ast.literal_eval(result_q[3])
type(d)

dict

In [ ]:
# Transformer le résultat en liste en utilisant une autre fonction de la bibliothèque utilisateur
r = [l for l in spqf.sparql_result_to_list(d)]
print(len(r))
r

## Les propriétés des économistes

La requête 8 de la base de données donne comme résultat la liste des propriétés regroupées des personnes classées dans l'ontologie DBPedia comme dbo:Economist ou associées avec des propriétés significatives au concept dbr:Economist

Le résultat de cette requête pose le problème de l'articulation entre les propriétés issues de la première méthode d'extraction des données de DBPedia (_http://dbpedia.org/property/..._) et celles de la nouvelle méthode basée sur une ontologie décrite explicititement (_http://dbpedia.org/ontology/..._).

Documentation à ce sujet:

* https://wiki.dbpedia.org/services-resources/ontology

* http://mappings.dbpedia.org/server/ontology/classes/Economist

* http://mappings.dbpedia.org/index.php/Main_Page


L'interrogation des propriétés dans l'espace de noms ontologie donne des données plus cohérentes. Elles sont parfois identiques aux premières, parfois elle demandent intégration. 

Il faut explorer les données pour comprendre l'information qu'elles contiennent et évaluer son utilité pour répondre aux questionnements.

[8 mai 2021] __Complément important__:
* la requête 8 a été complétée en ajoutant la classe (rdf:type) [Economist110043643](https://dbpedia.org/class/yago/Economist110043643) issue du projet [YAGO: A High-Quality Knowledge Base](https://yago-knowledge.org/)
  * https://yago-knowledge.org/schema
  * https://yago-knowledge.org/sparql
  * [YAGO (database)](https://en.wikipedia.org/wiki/YAGO_(database)) (Wikpedia)
* le nombre d'économistes a plus que doublé et des figures significatives, tel que [John Maynard Keynes](https://dbpedia.org/page/John_Maynard_Keynes) —qui n'est pas classé comme économiste dans l'ontolgie DBPedia, sont maintenant comprises : cf. les requêts 7 et 19 qui donnent l'effectif des économistes retrouvés

La leçon à tirer : l'inspection 'manuelle' des données et l'expertise de domaine sont essentielles en analyse de données